In [10]:
import pandas as pd

df = pd.read_parquet(
    "/home/darveter/Desktop/sdd2/Projects_backup/MLOps-traning/task-for-exam/Tasks4/fraud_features/feature_repo/data/fraud_transactions_fixed_new_fixed.parquet",
)

print(df.head())
print(df.columns)
print(df.dtypes)
print(df.shape)

   transaction_id               tx_datetime  customer_id  terminal_id  \
0               0 2019-08-22 06:51:03+00:00            0          711   
1               1 2019-08-22 05:10:37+00:00            0            0   
2               2 2019-08-22 19:05:33+00:00            0          753   
3               3 2019-08-22 07:21:33+00:00            0            0   
4               4 2019-08-22 09:06:17+00:00            1          981   

   tx_amount  tx_time_seconds  tx_time_days  tx_fraud  tx_fraud_scenario  
0      70.91            24663             0         0                  0  
1      90.55            18637             0         0                  0  
2      35.38            68733             0         0                  0  
3      80.41            26493             0         0                  0  
4     102.83            32777             0         0                  0  
Index(['transaction_id', 'tx_datetime', 'customer_id', 'terminal_id',
       'tx_amount', 'tx_time_seconds', 't

In [12]:
from feast import FeatureStore
import pandas as pd

# путь к каталогу feature_repo (можно абсолютный)
store = FeatureStore(repo_path="fraud_features/feature_repo")

# ───── entity_df ─────
entity_df = pd.DataFrame(
    {
        "customer_id": pd.Series([0, 1, 2, 3], dtype="int32"),
        "event_timestamp": pd.to_datetime("2019-08-24 00:00:00", utc=True),
    }
)

# ───── запрос ─────
training_df = (
    store.get_historical_features(
        entity_df=entity_df,
        features=[
            "tx_features:tx_amount",
            "tx_features:tx_time_seconds",
            "fraud_labels:tx_fraud",
            "tx_flags:is_large_tx",
        ],
    )
    .to_df()
)

print(training_df.head())


/home/darveter/Desktop/sdd2/Projects_backup/MLOps-traning/all-venv/feast/lib/python3.11/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


   customer_id           event_timestamp  tx_amount  tx_time_seconds  \
0            2 2019-08-24 00:00:00+00:00        NaN              NaN   
1            1 2019-08-24 00:00:00+00:00     102.83          32777.0   
2            3 2019-08-24 00:00:00+00:00      34.20          67285.0   
3            0 2019-08-24 00:00:00+00:00      35.38          68733.0   

   tx_fraud  is_large_tx  
0       NaN            0  
1       0.0            0  
2       0.0            0  
3       0.0            0  


In [10]:
import pandas as pd, pyarrow as pa, pyarrow.parquet as pq
from pathlib import Path

src = Path("fraud_features/feature_repo/data/fraud_transactions_fixed_new.parquet")
dst = src.with_stem(src.stem + "_fixed")              # fraud_transactions_fixed_new_fixed.parquet

df = pd.read_parquet(src)

# ──────────────────────────────────────────────
# 1. исправляем строки с часом «24»
# ──────────────────────────────────────────────
mask_24h = df["tx_datetime"].str.contains(r" 24:")
#   "2019‑08‑22 24:00:00"  →  "2019‑08‑23 00:00:00"
df.loc[mask_24h, "tx_datetime"] = (
    pd.to_datetime(
        df.loc[mask_24h, "tx_datetime"].str.replace(r" 24:", " 00:", regex=True),
        format="%Y-%m-%d %H:%M:%S",
        utc=True,
    )
    + pd.Timedelta(days=1)
)

# остальные строки конвертируем обычным способом
df.loc[~mask_24h, "tx_datetime"] = pd.to_datetime(
    df.loc[~mask_24h, "tx_datetime"],
    format="%Y-%m-%d %H:%M:%S",
    utc=True,
)

# ──────────────────────────────────────────────
# 2. сохраняем исправленный parquet
# ──────────────────────────────────────────────
pq.write_table(pa.Table.from_pandas(df), dst, compression="snappy")
print(f"✅  Новый файл записан: {dst}")


✅  Новый файл записан: fraud_features/feature_repo/data/fraud_transactions_fixed_new_fixed.parquet


In [ ]:
import pandas as pd

df = pd.read_parquet(
    "s3a://otus/clean/fraud_transactions_fixed_new.parquet",
    storage_options={
        "endpoint_url": "http://192.168.31.201:9000",
        "key": "admin",
        "secret": "password"
    }
)

print(df.head())
print(df.columns)
print(df.dtypes)
print(df.shape)

In [ ]:
# ────────────────────────────────────────
# 0. (Необязательно) задаём ключи MinIO
#    — если вы уже экспортировали AWS_ACCESS_KEY_ID/
#      AWS_SECRET_ACCESS_KEY в оболочке, блок можно
#      пропустить или закомментировать.
# ────────────────────────────────────────
import os
os.environ.setdefault("AWS_ACCESS_KEY_ID", "admin")
os.environ.setdefault("AWS_SECRET_ACCESS_KEY", "password")
os.environ.setdefault("AWS_ENDPOINT_URL", "http://192.168.31.201:9000")

# ───────────────────
# 1. Подключаем Feast
# ───────────────────
from feast import FeatureStore
import pandas as pd

store = FeatureStore(repo_path="fraud_features/feature_repo")  # путь до feature_repo

# ───────────────────
# 2. Формируем entity_df
# ───────────────────
# Допустим, берём снимок на 24 августа 2019 г. (дата из вашего датасета)
snap_time = pd.to_datetime("2019-08-24 00:00:00")

entity_df = pd.DataFrame(
    {
        "customer_id": [0, 1, 2, 3],
        "event_timestamp": [snap_time] * 4,      # одинаковая точка времени
    }
)

# ───────────────────
# 3. Запрашиваем исторические признаки
# ───────────────────
feature_list = [
    "tx_features:tx_amount",
    "tx_features:tx_time_seconds",
    "fraud_labels:tx_fraud",
    "tx_flags:is_large_tx",            # ← on‑demand FV
]

hist_df = (
    store.get_historical_features(
        entity_df=entity_df,
        features=feature_list,
    )
    .to_df()
    .sort_values("customer_id")
    .reset_index(drop=True)
)

# ───────────────────
# 4. Смотрим результат
# ───────────────────
print(hist_df.head())